<h1>Pesquisa em profundidade e pesquisa em largura em Python</h1>

<p>A teoria dos grafos e em particular o gráfico ADT (abstract data-type) é amplamente explorada e implementada no campo da Ciência da Computação e da Matemática. Consistindo em vértices (nós) e as arestas (opcionalmente direcionadas / ponderadas) que os conectam, a estrutura de dados é efetivamente capaz de representar e resolver muitos domínios de problemas. Uma das áreas mais populares do projeto de algoritmo neste espaço é o problema de verificar a existência ou o caminho (mais curto) entre dois ou mais vértices no gráfico. Propriedades como peso da aresta e direção são dois fatores que o designer do algoritmo pode levar em consideração. Nesta postagem, explorarei dois dos algoritmos mais simples disponíveis, pesquisa de profundidade e respiração para atingir os objetivos destacados abaixo:</p>
<ol>
    <li>Encontre todos os vértices em um componente conectado de vértices de assunto.</li>
    <li>Retorne todos os caminhos disponíveis entre dois vértices.</li>
    <li>E, no caso do BFS, retorna o caminho mais curto (comprimento medido pelo número de bordas do caminho).</li>
</ol>

<h1>O gráfico</h1>

<p>Para discutir claramente cada algoritmo, criei um grafo conectado com seis vértices e seis arestas incidentes. O gráfico resultante não é direcionado sem nenhuma ponderação de aresta atribuída, pois o comprimento será avaliado com base no número de arestas do caminho percorridas. Existem duas opções populares para representar um grafo, a primeira sendo uma matriz de adjacência (efetiva com grafos densos) e a segunda uma lista de adjacência (efetiva com grafos esparsos). Optei por implementar uma lista de adjacências que armazena cada nó em um dicionário junto com um conjunto contendo seus nós adjacentes. À medida que o gráfico não é direcionado, cada borda é armazenada em ambos os conjuntos adjacentes de nós incidentes.</p>

In [2]:
graph = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['BF': set(['C', 'E'])}
graph', 'F']),
         '

{'A': {'B', 'C'},
 'B': {'A', 'D', 'E'},
 'C': {'A', 'F'},
 'D': {'B'},
 'E': {'B', 'F'},
 'F': {'C', 'E'}}

<p>Olhando para a representação gráfica abaixo, você também notará a inclusão de um ciclo, pelas conexões adjacentes entre 'F' e 'C / E'. Isso foi incluído propositalmente para fornecer aos algoritmos a opção de retornar vários caminhos entre dois nós desejados.</p>

<img src="img.png">

<h1>Pesquisa em profundidade</h1>

<p>O primeiro algoritmo que irei discutir é a pesquisa em profundidade que, como o nome sugere, explora os vértices possíveis (de uma raiz fornecida) em cada ramificação antes de retroceder. Esta propriedade permite que o algoritmo seja implementado de forma sucinta nas formas iterativa e recursiva. Abaixo está uma lista das ações realizadas em cada visita a um nó.</p>

<ul>
    <li>Marque o vértice atual como sendo visitado.</li>
    <li>Explore cada vértice adjacente que não está incluído no conjunto visitado.</li>
</ul>

<h1>Componente Conectado</h1>

<p>A implementação abaixo usa a estrutura de dados da pilha para construir e retornar um conjunto de vértices que são acessíveis dentro do componente conectado do sujeito. Usando a sobrecarga do Python do operador de subtração para remover itens de um conjunto, somos capazes de adicionar apenas os vértices adjacentes não visitados.</p>

In [5]:
def dfs(graph, start):
    visited, stack = set(), [start] # ❌ Como set () não está ordenado, a ordem das visitas aos nós é perdida.
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
    return visited

dfs(graph, 'A') # {'E', 'D', 'F', 'A', 'C', 'B'} ❌ Resultado não codificado ou correto para algoritmo

{'A', 'B', 'C', 'D', 'E', 'F'}

In [6]:
# ✅ Fixo?

def dfs(graph, start):
    visited, stack = [], [start] # ✅ Lista de uso
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.append(vertex)
            stack.extend(graph[vertex] - set(visited)) # ✅ Converter para conjunto ()
    return visited

dfs(graph, 'A') # ['A', 'B', 'E', 'F', 'C', 'D'] # Saída correta

['A', 'B', 'E', 'F', 'C', 'D']

<p>A segunda implementação fornece a mesma funcionalidade que a primeira, entretanto, desta vez estamos usando a forma recursiva mais sucinta. Devido a uma pegadinha comum do Python com valores de parâmetro padrão sendo criados apenas uma vez, somos obrigados a criar um novo conjunto visitado em cada invocação do usuário. Outro detalhe da linguagem Python é que as variáveis ​​de função são passadas por referência, fazendo com que o conjunto mutável visitado não precise ser reatribuído a cada chamada recursiva.</p>

In [7]:
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set() # ❌ Como set () não está ordenado, a ordem das visitas aos nós é perdida.
    visited.add(start)
    for next in graph[start] - visited:
        dfs(graph, next, visited)
    return visited

dfs(graph, 'C') # {'E', 'D', 'F', 'A', 'C', 'B'} ❌ Não corresponde à saída e a saída está incorreta

{'A', 'B', 'C', 'D', 'E', 'F'}

In [8]:
# ✅ Fixo?

def dfs(graph, start, visited=[]): # ✅ Lista de uso
    visited.append(start)
    for next in graph[start] - set(visited): # ✅ Converter para conjunto ()
        dfs(graph, next, visited)
    return visited

dfs(graph, 'C') # ['C', 'F', 'E', 'B', 'A', 'D', 'A']

['C', 'F', 'E', 'B', 'D', 'A', 'A']

<h1>Caminhos</h1>

<p>Podemos ajustar ambas as implementações anteriores para retornar todos os caminhos possíveis entre um vértice inicial e objetivo. A implementação abaixo usa a estrutura de dados da pilha novamente para resolver o problema de forma iterativa, produzindo cada caminho possível quando localizamos a meta. Usar um gerador permite que o usuário calcule apenas a quantidade desejada de caminhos alternativos.</p>

In [9]:
def dfs_paths(graph, start, goal):
    stack = [(start, [start])]
    while stack:
        (vertex, path) = stack.pop()
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                stack.append((next, path + [next]))

list(dfs_paths(graph, 'A', 'F')) # [['A', 'C', 'F'], ['A', 'B', 'E', 'F']] ⚠️ Correto, mas a ordem é diferente

[['A', 'B', 'E', 'F'], ['A', 'C', 'F']]

<p>A implementação abaixo usa a abordagem recursiva chamando a adição "rendimento de" PEP380 para retornar os caminhos localizados invocados. Infelizmente, a versão de Pygments instalada no servidor neste momento não inclui a combinação de palavras-chave atualizada.</p>

In [10]:
def dfs_paths(graph, start, goal, path=None):
    if path is None:
        path = [start]
    if start == goal:
        yield path
    for next in graph[start] - set(path):
        yield from dfs_paths(graph, next, goal, path + [next])

list(dfs_paths(graph, 'C', 'F')) # [['C', 'F'], ['C', 'A', 'B', 'E', 'F']]

[['C', 'F'], ['C', 'A', 'B', 'E', 'F']]

<h1>Pesquisa em amplitude</h1>

<p>Um algoritmo alternativo chamado pesquisa Breath-First nos fornece a capacidade de retornar os mesmos resultados que o DFS, mas com a garantia adicional de retornar o caminho mais curto primeiro. Esse algoritmo é um pouco mais complicado de implementar de maneira recursiva, em vez de usar a estrutura de dados da fila, como tal, documentarei apenas a abordagem iterativa. As ações executadas por cada vértice explorado são as mesmas da implementação em profundidade, no entanto, substituir a pilha por uma fila explorará a amplitude da profundidade de um vértice antes de prosseguir. Este comportamento garante que o primeiro caminho localizado seja um dos caminhos mais curtos presentes, com base no número de arestas sendo o fator de custo.</p>

<p>Componente conectado Semelhante à implementação iterativa do DFS, a única alteração necessária é remover o próximo item do início da estrutura da lista em vez das pilhas por último.</p>

In [11]:
def bfs(graph, start):
    visited, queue = set(), [start]
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            visited.add(vertex)
            queue.extend(graph[vertex] - visited)
    return visited

bfs(graph, 'A') # {'B', 'C', 'A', 'F', 'D', 'E'} ❌❌❌

{'A', 'B', 'C', 'D', 'E', 'F'}

In [12]:
# ✅ Fixo?

def bfs(graph, start):
    visited, queue = [], [start] # ✅ Lista de uso
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            visited.append(vertex)
            queue.extend(graph[vertex] - set(visited)) # ✅ Converter lista em conjunto ()
    return visited

bfs(graph, 'A') # ['A', 'C', 'B', 'F', 'D', 'E'] # ✅

['A', 'C', 'B', 'F', 'D', 'E']

<h1>Caminhos</h1>

<p>Esta implementação pode novamente ser ligeiramente alterada para, em vez disso, retornar todos os caminhos possíveis entre dois vértices, o primeiro dos quais sendo um dos caminhos mais curtos.</p>

In [13]:
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
            else:
                queue.append((next, path + [next]))

list(bfs_paths(graph, 'A', 'F')) # [['A', 'C', 'F'], ['A', 'B', 'E', 'F']]

[['A', 'C', 'F'], ['A', 'B', 'E', 'F']]

<p>Sabendo que o caminho mais curto será retornado primeiro do método gerador de caminho BFS, podemos criar um método útil que simplesmente retorna o caminho mais curto encontrado ou 'Nenhum' se nenhum caminho existir. Como estamos usando um gerador, isso em teoria deve fornecer resultados de desempenho semelhantes como apenas quebrar e retornar o primeiro caminho correspondente na implementação do BFS.</p>

In [14]:
def shortest_path(graph, start, goal):
    try:
        return next(bfs_paths(graph, start, goal))
    except StopIteration:
        return None

shortest_path(graph, 'A', 'F') # ['A', 'C', 'F']

['A', 'C', 'F']